In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset

In [2]:
#Device

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [3]:
# Classes & Transform

class1 = 0  # T-shirt/top
class2 = 3  # Dress

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])



In [4]:
#Load Dataset

train_dataset_full = datasets.FashionMNIST(
    root="./data", train=True, download=True, transform=transform)

test_dataset_full = datasets.FashionMNIST(
    root="./data", train=False, download=True, transform=transform)

print("Training samples:", len(train_dataset_full))
print("Testing samples:", len(test_dataset_full))


100%|██████████| 26.4M/26.4M [00:02<00:00, 12.8MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 204kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.77MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 25.8MB/s]

Training samples: 60000
Testing samples: 10000


In [5]:
#Filter Two Classes

train_indices = [i for i, t in enumerate(train_dataset_full.targets)
                 if t in [class1, class2]]

test_indices = [i for i, t in enumerate(test_dataset_full.targets)
                if t in [class1, class2]]

train_dataset = Subset(train_dataset_full, train_indices)
test_dataset = Subset(test_dataset_full, test_indices)

print("Filtered training samples:", len(train_dataset))
print("Filtered testing samples:", len(test_dataset))


Filtered training samples: 12000
Filtered testing samples: 2000


In [6]:
#Target Mapping

def binary_target_transform(target):
    return 0 if target == class1 else 1

train_dataset_full.target_transform = binary_target_transform
test_dataset_full.target_transform = binary_target_transform


In [7]:
#DataLoaders

train_loader = DataLoader(train_dataset, batch_size=32,
                          shuffle=True, num_workers=0)

test_loader = DataLoader(test_dataset, batch_size=32,
                         shuffle=False, num_workers=0)


In [8]:
#Load Model


model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Freeze all layers
for param in model.parameters():
    param.requires_grad = False

# Unfreeze last convolution block (Fine-Tuning)
for param in model.layer4.parameters():
    param.requires_grad = True

# Replace final layer
model.fc = nn.Linear(model.fc.in_features, 2)

model = model.to(device)



Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 128MB/s]


In [9]:
#Loss & Optimizer

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)


In [10]:
epochs = 5

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total

    print(f"Epoch [{epoch+1}/5]")
    print(f"Training Loss: {epoch_loss:.4f}")
    print(f"Training Accuracy: {epoch_acc:.4f}")
    print("---------------------------")

Epoch [1/5]
Training Loss: 0.1127
Training Accuracy: 0.9557
---------------------------
Epoch [2/5]
Training Loss: 0.0388
Training Accuracy: 0.9871
---------------------------
Epoch [3/5]
Training Loss: 0.0209
Training Accuracy: 0.9928
---------------------------
Epoch [4/5]
Training Loss: 0.0106
Training Accuracy: 0.9971
---------------------------
Epoch [5/5]
Training Loss: 0.0090
Training Accuracy: 0.9964
---------------------------


In [11]:
model.eval()
correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        test_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

final_loss = test_loss / len(test_loader)
final_acc = correct / total

print("Test Results")
print("Test Loss:", round(final_loss,4))
print("Test Accuracy:", round(final_acc,4))

Test Results
Test Loss: 0.0922
Test Accuracy: 0.9765
